In [ ]:
import pandas as pd
from scipy.signal import detrend as sp_detrend
import matplotlib.pyplot as plt
%matplotlib inline
import os
import numpy as np
import requests


# Introduction to python for hydrologists &mdash; pandas

## Fourier analysis - so nerdy. so fun
https://youtu.be/spUNpyF58BY

## read in the daily values for the entire history of Lee's Ferry

In [ ]:
dv_url = 'http://waterservices.usgs.gov/nwis/dv/?format=rdb'
dv_url += '&sites={0}'.format('09380000')
#dv_url += '&startDT=2010-01-01'
dv_url += '&startDT=1880-01-01'
#dv_url += '&endDT=2018-01-17'
dv_url += '&parameterCd=00060'
print(dv_url)


In [ ]:
print ('reading data')
dv_file = requests.get(dv_url)
if not os.path.exists(os.path.join('data','pandas')):
    os.mkdir(os.path.join('data','pandas'))
with open(os.path.join('data','pandas','Lees Ferry.dat'), 'w') as ofp:
    for carp in dv_file:
        ofp.write(carp.decode())

In [ ]:
NWISfilename = os.path.join('data','pandas','Lees Ferry.dat')
reconnoiter = open(NWISfilename, 'r').readlines()
for i in np.arange(60):
    print (reconnoiter[i].rstrip())

In [ ]:
numhash = 0 #let's use the as the counter
for line in reconnoiter:
    if line.startswith('#'):
        numhash +=1
    else:
        break
        
print (numhash)

# Read in the time series

In [ ]:
colnames = reconnoiter[numhash].rstrip().split()
colnames[3] = 'Q'

df_all = pd.read_csv(os.path.join('data','pandas','Lees Ferry.dat'),
                      sep='\t',
                      skiprows = numhash+2,
                      names = colnames,
                      parse_dates = True,
                      index_col = 2)

## we have to detrend the data for `fft` to work properly. care why? read this: https://groups.google.com/forum/#!topic/comp.dsp/kYDZqetr_TU

## how do we do this? python is "batteries included" https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.detrend.html

## doesn't look different, but a linear trend was subtracted

In [ ]:
df_all['Q_detrend'] = sp_detrend(df_all.Q)

In [ ]:
N = len(df_all)
fig = plt.figure(figsize=(12,6))
ax = plt.subplot(3,1,1)
plt.plot(df_all.Q_detrend)
plt.title('detrended signal')
plt.grid()

plt.subplot(3,1,2)
plt.plot(range(len(df_all)),df_all.Q)
plt.title('Raw Signal')


plt.subplot(3,1,3)
plt.plot(range(len(df_all)),df_all.Q-df_all.Q_detrend)
plt.title('difference')


plt.tight_layout()

In [ ]:
N = len(df_all)
yf = np.fft.fft(df_all.Q_detrend)
yf = np.abs(yf[:int(N/2)])
# get the right frequency 
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.fft.fftfreq.html#numpy.fft.fftfreq
d = 1. # day
f = np.fft.fftfreq(N,d)[:int(N/2)]
per = 1./f # days
fig = plt.figure(figsize=(12,6))
ax = plt.subplot(2,1,1)
plt.plot(per, yf)
plt.xscale('log')

top4=np.argsort(yf)[-4:]
j=0.6
for i in top4:
    plt.plot([per[i],per[i]],[0,np.max(yf)],'r:')
    plt.text(per[i],j*np.max(yf),'{0:.2f}'.format(per[i]))
    j+=0.1

plt.title('Period Spectrum')
plt.grid()
ax.set_xlabel('Period (days)')
plt.xlim([0, 1e4])


plt.subplot(2,1,2)
plt.plot(range(len(df_all)),df_all.Q)
plt.title('Raw Signal')
plt.tight_layout()

## look only at years prior to Glen Canyon dam filling

In [ ]:
df = df_all.loc[df_all.index.year<1963].copy()
N = len(df)
d = 1. # day
f = np.fft.fftfreq(N,d)[:int(N/2)]
per = 1./f # days

yf_predam = np.fft.fft(sp_detrend(df.Q))
yf_predam = np.abs(yf_predam[:int(N/2)])

fig = plt.figure(figsize=(12,4))
plt.subplot(2,1,1)
plt.plot(per,yf_predam)
top4=np.argsort(yf_predam)[-4:]
j=0.6
for i in top4:
    plt.plot([per[i],per[i]],[0,np.max(yf_predam)],'r:')
    plt.text(per[i],j*np.max(yf_predam),'{0:.2f}'.format(per[i]))
    j+=0.1

plt.xscale('log')

plt.grid()
plt.xlim([0,1e4])
plt.subplot(2,1,2)
plt.plot(range(len(df)),df.Q)

## did things change after the dam filled up in 1963?

In [ ]:
df = df_all.copy()
df = df.loc[df.index.year>1963]
N = len(df)
d = 1. # day
f = np.fft.fftfreq(N,d)[:int(N/2)]
per = 1./f # days
yf_postdam = np.fft.fft(sp_detrend(df.Q))
yf_postdam = np.abs(yf_postdam[:int(N/2)])
fig = plt.figure(figsize=(12,4))
plt.subplot(2,1,1)
plt.plot(per,yf_postdam)

top10=np.argsort(yf_postdam)[-10:]
j=0.2
for i in top10:
    plt.plot([per[i],per[i]],[0,np.max(yf_postdam)],'r:')
    plt.text(per[i],j*np.max(yf_postdam),'{0:.2f}'.format(per[i]))
    j+=0.1

plt.xscale('log')

plt.subplot(2,1,2)
plt.plot(range(len(df)),df.Q)
#plt.xlim([0,2500])

## in the heady days of grunge?

In [ ]:
df = df_all.copy()
df = df.loc[(df.index.year>=1993) & (df.index.year<=1996)]
N = len(df)
d = 1. # day
f = np.fft.fftfreq(N,d)[:int(N/2)]
per = 1./f # days
yf_postdam = np.fft.fft(sp_detrend(df.Q))
yf_postdam = np.abs(yf_postdam[:int(N/2)])
fig = plt.figure(figsize=(12,4))
plt.subplot(2,1,1)
plt.plot(per,yf_postdam)

top10=np.argsort(yf_postdam)[-10:]
j=0.2
for i in top10:
    plt.plot([per[i],per[i]],[0,np.max(yf_postdam)],'r:')
    plt.text(per[i],j*np.max(yf_postdam),'{0:.2f}'.format(per[i]))
    j+=0.1

plt.xscale('log')

plt.subplot(2,1,2)
plt.plot(range(len(df)),df.Q)


## what's up with a dominant frequency of 7 days?

## let's check out a couple weeks of the signal

In [ ]:
plt.figure(figsize=(16,4))
cfreq = 40
df.iloc[:cfreq].Q.plot(rot=45, grid=True)
plt.xticks(df.iloc[:cfreq].index,df.iloc[:cfreq].index.weekday_name);

## looks like power management. similar these days?

In [ ]:
df = df_all.copy()
df = df.loc[(df.index.year>=2017) & (df.index<'2018-01-1')]
N = len(df)
d = 1. # day
f = np.fft.fftfreq(N,d)[:int(N/2)]
per = 1./f # days
yf_postdam = np.fft.fft(sp_detrend(df.Q))
yf_postdam = np.abs(yf_postdam[:int(N/2)])
fig = plt.figure(figsize=(12,4))
plt.subplot(2,1,1)
plt.plot(per,yf_postdam)

top10=np.argsort(yf_postdam)[-10:]
j=0.2
for i in top10:
    plt.plot([per[i],per[i]],[0,np.max(yf_postdam)],'r:')
    plt.text(per[i],j*np.max(yf_postdam),'{0:.2f}'.format(per[i]))
    j+=0.1

plt.xscale('log')

plt.subplot(2,1,2)
plt.plot(range(len(df)),df.Q)

In [ ]:
plt.figure(figsize=(16,4))
cfreq = 30
df.iloc[:cfreq].Q.plot(rot=45, grid=True)
plt.xticks(df.iloc[:cfreq].index,df.iloc[:cfreq].index.weekday_name);